#### Imports
<span style="font-family: Arial; font-size: 11pt;">
Der folgende Code enthält die Imports der vergangenen Saison (21/22), die für dieses Data Science-Projekt verwendet werden:

- `requests`: Diese Bibliothek ermöglicht das Senden von HTTP-Anfragen an Webseiten und die Verarbeitung der erhaltenen Daten.

- `BeautifulSoup` aus `bs4`: Diese Bibliothek wird verwendet, um HTML- oder XML-Dokumente zu parsen. Sie ermöglicht das Durchsuchen von Webseiten und das Extrahieren spezifischer Daten.

- `psycopg2`: Diese Bibliothek dient als Schnittstelle zur PostgreSQL-Datenbank. Sie ermöglicht das Herstellen von Verbindungen zur Datenbank und das Ausführen von Abfragen sowie andere Datenbankoperationen.</span>

In [ ]:
import requests
from bs4 import BeautifulSoup
import psycopg2

#### Kontrollvariable
<span style="font-family: Arial; font-size: 11pt;">
In diesem Codeabschnitt haben wir die Kontrollvariable `conn_Ok` mit dem anfänglichen Wert `True` implementiert, mit der anschliessend überprüft werden soll, ob die Verbindung zur DB steht</span>

In [ ]:
# Variablen
conn_Ok = True

#### DB-Verbindung
<span style="font-family: Arial; font-size: 11pt;">
Es wird mit einer Try-Except Methode versucht, eine Verbindung zur PostgreSQL-Datenbank herzustellen.
- Falls die Verbindung erfolgreich hergestellt werden kann, wird ein Cursor-Objekt erstellt und die Variable `conn_Ok` bleibt auf `True`.

Falls ein Verbindungsfehler auftritt, wird eine Fehlermeldung ausgegeben und die Variable `conn_Ok` auf `False` gesetzt.
</span>

In [ ]:
# Check if connection to DB possible
try:
    connection = psycopg2.connect(
        dbname='PSQL_ADSFS2023Gruppe15',
        user='ADSFS2023Gruppe15',
        password='ADS_FS_2023_G15!?',
        host='localhost',
        port='5432'
    )

    cursor = connection.cursor()

except (Exception, psycopg2.Error) as error:
        print('Fehler beim Verbinden mit der PostgreSQL-Datenbank:', error)
        conn_Ok = False

#### Datenbereinigung
<span style="font-family: Arial; font-size: 11pt;">
Der folgende Code definiert die Funktion get_team_id, die die Team-ID anhand des Teamnamens aus der PostgreSQL-Datenbank abruft.

- Der übergebene `team_name` wird auf spezifische Teamnamen überprüft und entsprechend angepasst, um den erwarteten Namen in der Datenbank zu verwenden.
- Es wird eine SQL-Abfrage ausgeführt, um die Team-ID aus der Datenbank abzurufen.
- Das Ergebnis der Abfrage wird überprüft und entweder die Team-ID zurückgegeben oder eine Fehlermeldung ausgegeben.
</span>

In [ ]:
def get_team_id(team_name):
    #Alle Teamnamen von Kicker.de anpassen, damit diese mit den Teamnamen in unserer Tabelle übereinstimmen
    if team_name == 'Gladbach':
        team_name = 'Borussia Mönchengladbach'
    if team_name == '%Gladbach':
        team_name = 'Borussia Mönchengladbach'    
    elif team_name == 'Hoffenheim':
        team_name = 'TSG 1899 Hoffenheim'
    elif team_name == '%Bremen':
        team_name = 'Werder Bremen'
    elif team_name == '%Hoffenheim':
        team_name = 'TSG 1899 Hoffenheim'
    elif team_name == '%Bremen':
        team_name = 'Werder Bremen'
    elif team_name == '%Bochum':
        team_name = 'VfL Bochum'
    elif team_name == '%Dortmund':
        team_name = 'Borussia Dortmund'                
    elif team_name == '%Bayern':
        team_name = 'FC Bayern München'
    elif team_name == '%Bayern Munich':
        team_name = 'FC Bayern München'        
    elif team_name == '%Frankfurt':
        team_name = 'Eintracht Frankfurt'                
    elif team_name == 'Köln':
        team_name = '1. FC Köln'
    elif team_name == '%Koln':
        team_name = '1. FC Köln'    
    elif team_name == 'Union':
        team_name = '1. FC Union Berlin'                
    elif team_name == '%Mainz%': #Mainz 05
        team_name = '1. FSV Mainz 05'
    elif team_name == '%Hertha':
        team_name = 'Hertha BSC'
    elif team_name == '%Freiburg':
        team_name = 'SC Freiburg'
    elif team_name == '%Stuttgart':
        team_name = 'VfB Stuttgart'                
    elif team_name == 'Leverkusen':
        team_name = 'Bayer Leverkusen'
    elif team_name == '%Leverkusen':
        team_name = 'Bayer Leverkusen'        
    elif team_name == 'Leipzig':
        team_name = 'RB Leipzig'         
    elif team_name == '%Wolfsburg':
        team_name = 'VfL Wolfsburg'
    elif team_name == '%Augsburg':
        team_name = 'FC Augsburg'              

#Abgestiegen: Fürht und Bielefeld > ignorieren einbauen nötig? Fehlt nun einfach hier
   


#Alle Teamnamen von whoscored.com anpassen, damit diese mit den Teamnamen in unserer Tabelle übereinstimmen

        
    # Vergleicht Team Namen und sucht nach ID
    cursor.execute("SELECT team_id FROM bundesliga_mannschaften WHERE mannschaft = %s", (team_name,))
    result = cursor.fetchone()[0]

    if result:
        return result
    else:
        print(f'Keine Übereinstimmung für {team_name} gefunden')


#### Laufleistung
<span style="font-family: Arial; font-size: 11pt;">

Der folgende Codeabschnitt durchläuft eine Schleife von 1 bis 34 (Spieltage) und ruft Daten von einer Webseite ab.

- Eine URL wird für jeden Spieltag generiert, um Daten von einer bestimmten Webseite abzurufen.
- Eine HTTP-Anfrage wird an die URL gesendet, um die Webseite abzurufen.
- Wenn die Antwort erfolgreich ist (200), werden relevante Informationen wie Mannschaftsnamen und Schüsse aufs Tor von der HTML-Seite extrahiert.
- Ein SQL-INSERT-Statement wird erstellt, um die Daten (Spieltag, Team-ID und Laufleistung) in der Datenbank zu speichern.
- Die Änderungen werden in der Datenbank gespeichert.
- Bei auftretenden Fehlern wird die Transaktion rückgängig gemacht und eine entsprechende Fehlermeldung ausgegeben.
</span>

In [ ]:
for matchday in range(1, 34):
    url = f'https://www.kicker.de/bundesliga/laufleistung/2021-22/{matchday}'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        team_elements = soup.find_all('td', class_='kick__table--ranking__teamname kick__table__cell--multiline kick__t__a__l')
        distance_elements = soup.find_all('td', class_='kick__table--ranking__master')
        try:
            for team_element, distance_element in zip(team_elements, distance_elements):
                team_name = team_element.text.strip()
                distance_total = distance_element.text.strip()

                insert_data_query = '''
                    INSERT INTO bundesliga_distance_stats (matchday, team_id, distance_total)
                    VALUES (%s, %s, %s);
                    '''
                team_id = get_team_id(team_name)
            
                cursor.execute(insert_data_query, (matchday, team_id, float(distance_total.replace(',', '.'))))
                            
                connection.commit()
        except Exception as e:
            connection.rollback
            print(f'Fehler beim Einfügen der Daten für Spieltag {matchday} und Laufleistung:', str(e))

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Laufleistung')

#### Summierte Daten
<span style="font-family: Arial; font-size: 11pt;">

Aufgrund der einerseits eingeschränkten Verfügbarkeit aller Daten von vergangenen Saisons und der geringen Aussagekraft für die Folgesaison werden hier die Daten als Total von der vorherigen Saison importiert. Der folgende Codeabschnitt ruft die Daten von einer Webseite ab.

- Eine URL wird für jeden Spieltag generiert, um Daten von einer bestimmten Webseite abzurufen.
- Eine HTTP-Anfrage wird an die URL gesendet, um die Webseite abzurufen.
- Wenn die Antwort erfolgreich ist (200), werden relevante Informationen wie Mannschaftsnamen und Schüsse aufs Tor von der HTML-Seite extrahiert.
- Ein SQL-INSERT-Statement wird erstellt, um die Daten (Spieltag, Team-ID und Eckbälle) in der Datenbank zu speichern.
- Die Änderungen werden in der Datenbank gespeichert.
- Bei auftretenden Fehlern wird die Transaktion rückgängig gemacht und eine entsprechende Fehlermeldung ausgegeben.
</span>

In [ ]:
url = 'https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/8667/Stages/19862/TeamStatistics/Germany-Bundesliga-2021-2022'
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    team_elements = soup.find_all('td', class_='col12-lg-2 col12-m-3 col12-s-4 col12-xs-5 grid-abs overflow-text')
    goals_elements = soup.find_all('td', class_='goal')
    shots_elements = soup.find_all('td', class_='shotsPerGame')
    possession_elements = soup.find_all('td', class_='possession')
    pass_elements = soup.find_all('td', class_='passSuccess')
    try:
        for team_element, goals_element, shots_element, possession_element, pass_element in zip(team_elements, goals_elements, shots_elements, possession_elements, pass_elements):
            team_name = team_element.text.strip()
            goals_total = goals_element.text.strip()
            shots_total = shots_element.text.strip()
            touches_total = possession_element.text.strip()
            pass_percentage = pass_element.text.strip()
    except Exception as e:
        print(f"An error occurred: {e}")
